<a href="https://colab.research.google.com/github/garrettfarley9/philly-real-estate-model/blob/data_cleaning_working/Philly_Real_estate_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Good house or nah? Program
>Sub functions
1. Rent (linear regressor?) predictor model (could be continuous tree)
  input - Address GIS
   - 3-5 comps from same zip / block sales in past 6mo - 1yr
    -edge case where a house is on the edge of a zip / block
   - machine learning predictor model for rent
  output - rent per sqft prediction

2. Appreciation predictor (fair market value)
  input - zip
    - use HUD data to find appreciation over 10 years
    - calculate appreciation for zip code
  output - appreciation value over 10 years

3. Iterate through all houses
  input - zip / block
    - use the Rent function to find houses
    - calculate IRR with assumptions and calculations with appreciation
  output - all houses that meet IRR criteria ranked by highest return

>Variables
>>Assumptions
1. Hold period - 120 months
2. Occupancy - 11/12 months/year
3. Annual growth rate - 4%
4. Maintenance - 10% of rent
5. Closing cost - 6%-8%
6. Money down - 20%
7. interest rate - market
>>Simple Calcuations (baby functions) from data/knowledge/input
1. tax
2. Mortgage cost

>Main function(data, house address, zip, *calculating_needs):
 1. Import city gis property data
 1.5 Run data cleaning pipe
 2. Choose zip / block
 3. Rent predictor model
 3.5 Calculate appreciation by zip
 4. Calculate IRR with assumptions and calculations
 5. Compare to threshold (15%) / Some profit number
 6. Output yes/no (maybe on a 0-1 scale?) and why
 6.5 Output as ranked list
 >(sneaky logistic regression application here?)

>Data Vis
1. map hued house by point
2. aggregate by block
'''

In [ ]:
'''
Perfect world data:
Dataset with coordinates for parcel shapes in a city. We have zip code or even block id to compare comps in the same zip code or block.
We have sale price data for recent purchases to get approx house price. We have rent data for comps to get approx rent.
We have robust data about what impacts the growth of the house value.
We have good assumptions/calculations of tax / mortgage / occupancy, what have you.


Found Philly datasets:
Property Assessments
https://data.phila.gov/visualizations/property-assessments
Info about houses
>Bed
>Bath
>Heating

Parcel shapes
https://opendataphilly.org/datasets/building-footprints/
>GIS Shapes

Real Estate Transfers
https://data.phila.gov/visualizations/real-estate-transfers
>When sold
>Sold for how much
>Can use differences to measure growth over time?
'''

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp


In [ ]:
# Constants
hold_period = 120
occupancy = 0.92
appreciation_estimated =  0.04

# >>Simple Calcuations (baby functions) from data/knowledge/input
def tax_func(house_value = 120000, percentage_tax = .015):
  return house_value * percentage_tax

def mortgage_func(interest_rate = .068, principal = 120000, term = 360):
  return principal * (interest_rate / 12) * (1 + (interest_rate / 12)) ** term

def maintenance_cost(rent = 1000):
  return rent / 10

def closing_cost(house_value = 120000, appreciation_estimated, hold_period):
  pred_value = house_value * ((1+appreciation_estimated) ** (hold_period / 12))
  return pred_value * 0.08

def money_down(house_value = 120000, percentage_down = 0.20):
  return house_value * percentage_down
'''
def interest_rate():

interest_rate = market

def meta_calc(house_value, percentage_down, appreciation_estimated, hold_period, term, interest_rate,
              rent, maintenance_cost, percentage_tax):
  tax_func()
  mortgage_func()
  maintenance_cost()
  closing_cost()
  money_down()
  return
'''

In [7]:
df = pd.read_csv('/content/opa_properties_public.csv')
df.head()


/tmp/ipython-input-2483399547.py:1: DtypeWarning: Columns (3,15,24,26,75) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/opa_properties_public.csv')


,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,...,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape,objectid
0,2024-05-30 21:05:37-04:00,NaN,20'W 54TH ST,54481181,O30,ROW 2 STY MASONRY,13,VACANT LAND - RESIDENTIAL,85.0,NaN,...,I,NaN,NaN,19143.0,RSA5,1001174389,NaN,NaN,SRID=2272;POINT(2674859.57342748 236424.8827173),708975581
1,2025-10-20 19:37:05-04:00,NaN,"259'8 1/2"" N ROBERTS",54481181,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,205.0,NaN,...,I,1940.0,Y,19144.0,RM1,1001253994,22.0,ROW TYPICAL,SRID=2272;POINT(2694035.57903865 261544.28162538),708975582
2,2025-10-20 19:39:51-04:00,NaN,162' N OF PACIFIC,54481181,U30,ROW CONV/APT 2 STY MASON,2,MULTI FAMILY,202.0,NaN,...,NaN,1930.0,NaN,19140.0,CMX2,1001631660,32.0,TWIN CONVENTIONAL,SRID=2272;POINT(2694066.92051139 257138.55487254),708975583
3,2025-10-20 19:21:34-04:00,NaN,40' W HOLLY ST,54481181,O50,ROW 3 STY MASONRY,1,SINGLE FAMILY,107.0,NaN,...,0,1915.0,Y,19104.0,RSA5,1001412532,24.0,ROW PORCH FRONT,SRID=2272;POINT(2681159.6195544 241552.48401064),708975584
4,2025-10-20 19:18:16-04:00,NaN,21' W HOLLY ST,54481181,O50,ROW 3 STY MASONRY,1,SINGLE FAMILY,107.0,NaN,...,I,1915.0,Y,19104.0,RSA5,1001412530,24.0,ROW PORCH FRONT,SRID=2272;POINT(2681178.8045554 241545.58769897),708975585


In [10]:
df.rename(columns={'mailing_street': 'address'}, inplace=True)
df.columns

Index(['assessment_date', 'basements', 'beginning_point', 'book_and_page',
       'building_code', 'building_code_description', 'category_code',
       'category_code_description', 'census_tract', 'central_air',
       'cross_reference', 'date_exterior_condition', 'depth',
       'exempt_building', 'exempt_land', 'exterior_condition', 'fireplaces',
       'frontage', 'fuel', 'garage_spaces', 'garage_type',
       'general_construction', 'geographic_ward', 'homestead_exemption',
       'house_extension', 'house_number', 'interior_condition', 'location',
       'mailing_address_1', 'mailing_address_2', 'mailing_care_of',
       'mailing_city_state', 'address', 'mailing_zip', 'market_value',
       'market_value_date', 'number_of_bathrooms', 'number_of_bedrooms',
       'number_of_rooms', 'number_stories', 'off_street_open',
       'other_building', 'owner_1', 'owner_2', 'parcel_number', 'parcel_shape',
       'quality_grade', 'recording_date', 'registry_number', 'sale_date',
       'sale

In [9]:
# df.shape
df_mini = pd.DataFrame.sample(df, 5000)
df_mini.shape
df_mini.to_csv('opa_5000.csv', index=False)

In [10]:
test = pd.read_csv('/content/opa_5000.csv')
# test.head()


,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,...,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape,objectid
0,2024-05-30 21:05:37-04:00,NaN,270' W 5TH ST,51547634.0,B58,DET W/GAR 3 STY STONE,1,SINGLE FAMILY,269.0,NaN,...,I,1930.0,Y,19126.0,RSA2,1001672791,07,CONVENTIONAL,SRID=2272;POINT(2702408.04392098 273332.97556764),709347859
1,2024-05-30 21:05:37-04:00,D,69' W UNION ST,50531867.0,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,108.0,NaN,...,I,1930.0,Y,19104.0,RM1,1001382250,22.0,ROW TYPICAL,SRID=2272;POINT(2682756.14621697 239840.7636808),709400815
2,2024-05-30 21:05:37-04:00,NaN,"155'6"" S GROVERS AVE",40294.0,H30,SEMI/DET 2 STY MASONRY,1,SINGLE FAMILY,56.0,NaN,...,I,1925.0,Y,19153.0,RSA5,1001676588,32,TWIN CONVENTIONAL,SRID=2272;POINT(2670765.47990964 216488.72606738),709501717
3,2024-05-30 21:05:37-04:00,NaN,"57'6"" N OF PRICE ST",1316051.0,Y50,STR/OFF+APT 3 STY MASONRY,3,MIXED USE,246.0,NaN,...,I,1900.0,NaN,19144.0,CMX2.5,1001237755,821.0,ROW MIXED-COM/RES-BLT AS COM,SRID=2272;POINT(2689266.00950956 266895.84769388),709485393
4,2024-05-30 21:05:37-04:00,NaN,78' N OF SPRUCE ST,37192.0,590,RES CONDO 5+ STY MASONRY,1,SINGLE FAMILY,10.0,NaN,...,I,1985.0,NaN,19106.0,RMX3,1001599518,41,NaN,SRID=2272;POINT(2697148.64925607 233915.13544022),709473666


In [4]:
df_foot = pd.read_csv('/content/LI_BUILDING_FOOTPRINTS.csv')
df_foot.head()

,objectid,bin,fcode,address,building_n,base_eleva,approx_hgt,max_hgt,parcel_id_,parcel_i_1,dor_altern,square_ft,Shape__Are,Shape__Len
0,68612981,1000001,1810,1501 N 18TH ST,NaN,104.2,25,32.562981,319860,PWD,NaN,617,97.753906,40.383631
1,68612982,1000002,1810,1501 N 18TH ST,NaN,103.3,26,28.700851,319860,PWD,NaN,629,99.640625,40.691884
2,68612983,1000003,1810,1501 N 18TH ST,NaN,103.9,26,27.972899,319860,PWD,NaN,608,96.296875,40.145485
3,68612984,1000004,1810,1501 N 18TH ST,NaN,103.6,26,28.342124,319860,PWD,NaN,623,98.699219,40.543015
4,68612985,1000005,1810,1501 N 18TH ST,NaN,108.0,27,29.782007,319860,PWD,NaN,643,101.898438,42.790290


In [11]:
df_1 = pd.read_csv('/content/LI_BUILDING_FOOTPRINTS.csv')
# df_2 = pd.read_csv('/content/opa_properties_public.csv')
df_merge = pd.merge(df_1, df, how = 'inner', on='address')


,objectid_x,bin,fcode,address,building_n,base_eleva,approx_hgt,max_hgt,parcel_id_,parcel_i_1,...,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,shape,objectid_y
0,68613006,1000026,1810,1543 N BOUVIER ST,NaN,107.5,35,36.836128,9099,PWD,...,I,1915.0,Y,19121.0,RSA5,1001096594,22.0,ROW TYPICAL,SRID=2272;POINT(2693217.75226223 245289.82692963),709235185
1,68613007,1000027,1810,1541 N BOUVIER ST,NaN,107.7,35,36.523924,420322,PWD,...,I,1915.0,Y,19121.0,RSA5,1001096593,22.0,ROW TYPICAL,SRID=2272;POINT(2693214.31329273 245275.38634172),709467677
2,68613008,1000028,1810,1545 N BOUVIER ST,NaN,107.2,34,39.993078,316172,PWD,...,I,1915.0,Y,19121.0,RSA5,1001096595,22,ROW TYPICAL,SRID=2272;POINT(2693219.70501423 245306.32525621),709489997
3,68613009,1000029,1810,1539 N BOUVIER ST,NaN,107.8,35,35.987901,111888,PWD,...,I,1920.0,Y,19121.0,RSA5,1001096592,22,ROW TYPICAL,SRID=2272;POINT(2693212.02064639 245261.64358705),709470510
4,68613010,1000030,1810,1516 N 17TH ST,NaN,107.4,40,44.049959,372729,PWD,...,I,1915.0,Y,19121.0,RM1,1001624914,36.0,TWIN OLD STYLE,SRID=2272;POINT(2693288.57462715 245119.90108839),708977838


In [13]:
df_merge.shape
df.to_csv('merge_set.csv', index=False)

In [ ]:
'''
Feature Reduction on Data
-llbuilding footprints can segment/bin by building height?

Future Notes
-there are good geospatial algos that we can play around with when we want to make
the comp algo.



Basement codes
	0. None – Indicates no basement.
A. Full Finished – Occupies the entire area under the first floor.
B. Full Semi-Finished – Could have some finish to include a floor covering,
and ceiling. It looks more like a living area rather than a basement.
C. Full Unfinished – Is a typical basement with unfinished concrete floor,
either rubble stone or cement over stone or concrete walls and would have
exposed wood joist ceilings.
D. Full – Unknown Finish
E. Partial Finished – Occupies a portion under the first floor. Be careful of
areas under sheds and porches. If there is a garage at basement level then it is a
partial basement.
F. Partial Semi-Finished – One or more finished areas.
G. Partial Unfinished
H. Partial - Unknown Finish
I. Unknown Size - Finished
J. Unknown Size - Unfinished

Category Codes
1. Residential
2. Hotels and Apartments
3. Store with Dwelling
4. Commercial
5. Industrial
6. Vacant Land